### Picks the best performing symbols based on parameter settings<br> - Get dates in df_close_clean that are missing in df_picks<br> - Calculate days_to_drop from df_close_clean s.t. the last date in df_close_clean is the missing date in df_picks<br> - Get picks for the missing dates


In [1]:
def compare_lists(list_a, list_b):
  """Compares two lists and returns a list of values that are in list A but not in list B.

  Args:
    list_a: A list of objects.
    list_b: A list of objects.

  Returns:
    A list of values that are in list_a but not in list_b.
  """

  list_difference = []

  for item in list_a:
    if item not in list_b:
      list_difference.append(item)

  return list_difference

In [2]:
import sys
import pandas as pd
import itertools
from yf_utils import random_slices, lookback_slices
from yf_utils import rank_perf, grp_tuples_sort_sum, top_set_sym_freq_cnt
from yf_utils import best_perf_syms_sets_lookback_slices
from myUtils import pickle_load, pickle_dump

pd.set_option("display.max_rows", 20)
# pd.set_option("display.max_columns", 11)
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 26)
# pd.set_option("display.width", 280)
pd.set_option("display.width", 500)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"

### Set parameters below

In [3]:
### Set parameters ######################################################################
# fp_df_eval_results = f'df_eval_results_{run_type}'
fp_df_picks = f"df_picks"

verbose = False  # True prints more output
# verbose = True  # True prints more output

store_results = True

# n_samples is the number of random samples of df chunks s.t.
#   len(df) = max_days_lookbacks. For getting the current picks
#   n_samples is always 1 since more samples will always yield
#   the same df chunk
n_samples = 1  # only need 1 sample to get the current picks

# for training, the number of days to lookback from iloc max-lookback iloc_end_train
l_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

# e.g sort from [[60, 30, 120], [15, 60, 30, 120]] to [[30, 60, 120], [15, 30, 60, 120]] 
l_sorted_days_lookbacks = []
for days_lookbacks in l_days_lookbacks:
    l_sorted_days_lookbacks.append(sorted(days_lookbacks))

# number of days from iloc_end_train are used to evaluate effectiveness of the training
days_eval = 0

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20

# slice starts and ends for selecting the best performing symbols
syms_start = 0
syms_end = 10
#########################################################################

print(f"verbose : {verbose }")
print(f"store_results: {store_results}")
print(f"n_samples: {n_samples}")
print(f"l_sorted_days_lookbacks: {l_sorted_days_lookbacks}")
print(f"days_eval: {days_eval}")
print(f"n_top_syms: {n_top_syms}")
print(f"syms_start: {syms_start}")
print(f"syms_end: {syms_end}")
print(f"fp_df_picks: {fp_df_picks}\n\n")

verbose : False
store_results: True
n_samples: 1
l_sorted_days_lookbacks: [[30, 60, 120], [15, 30, 60, 120]]
days_eval: 0
n_top_syms: 20
syms_start: 0
syms_end: 10
fp_df_picks: df_picks




### Load past picks

In [4]:
df_picks = pickle_load(path_data_dump, fp_df_picks)
# drop duplicates
df_picks = df_picks.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
# sort, most recent date is first
df_picks = df_picks.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
# re-index
df_picks = df_picks.reset_index(drop=True)
# save results
pickle_dump(df_picks, path_data_dump, fp_df_picks)
print(f'df_picks, len({len(df_picks)}):\n{df_picks}')

df_picks, len(388):
    date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14 sym_freq_13 sym_freq_12      sym_freq_11      sym_freq_10       sym_freq_9       sym_freq_8              sym_freq_7                 sym_freq_6                 sym_freq_5                 sym_freq_4                 sym_freq_3 sym_freq_2
0          2023-12-19                 120      [30, 60, 120]          []          []          []          []               []               []               []   ['GPS', 'SHV']                ['FTSM']                         []  ['ANF', 'HIBB', 'SQ', ...  ['AMKR', 'LRN', 'MARA'...  ['BA', 'BPMC', 'BTC-US...         []
1          2023-12-19                 120  [15, 30, 60, 120]          []          []          []          []               []          ['SHV']               []          ['GPS']         ['FTSM', 'MBI']  ['HA', 'MARA', 'WOR', ...  ['ANF', 'FFWM', 'HIBB'...  ['AMKR', 'LOB', 'LRN',...                     ['BA']         []
2          2023-12-1

In [5]:
df_picks

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-12-19,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['GPS', 'SHV']",['FTSM'],[],"['ANF', 'HIBB', 'SQ', ...","['AMKR', 'LRN', 'MARA'...","['BA', 'BPMC', 'BTC-US...",[]
1,2023-12-19,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],[],['GPS'],"['FTSM', 'MBI']","['HA', 'MARA', 'WOR', ...","['ANF', 'FFWM', 'HIBB'...","['AMKR', 'LOB', 'LRN',...",['BA'],[]
2,2023-12-18,120,"[30, 60, 120]",[],[],[],[],[],[],[],['SHV'],"['FTSM', 'GPS']",['X'],"['ANF', 'HIBB', 'LRN']","['AMKR', 'KKR', 'MARA'...","['BPMC', 'BTC-USD', 'C...",[]
3,2023-12-18,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],['X'],[],"['FTSM', 'GPS', 'MBI']","['HA', 'MARA']","['ANF', 'HIBB', 'LRN',...","['AMKR', 'CG', 'KKR', ...","['BPMC', 'BTC-USD']",[]
4,2023-12-15,120,"[30, 60, 120]",[],[],[],[],[],[],[],['SHV'],"['FTSM', 'GPS']",[],"['ANF', 'HIBB', 'LRN']","['MARA', 'MBI', 'SQ']","['AMKR', 'AXGN', 'BPMC...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,2023-03-17,120,"[15, 30, 60, 120]",[],[],[],[],['SHV'],[],"['FTSM', 'GE']",[],"['NVDA', 'SGEN']","['ANET', 'FCN', 'GBTC'...","['ACLS', 'FSLR', 'HY',...","['AMD', 'ATCO', 'MLR']",['AJRD'],[]
384,2023-03-16,120,"[30, 60, 120]",[],[],[],[],[],[],"['FTSM', 'SHV']",[],[],"['LNTH', 'SGEN', 'WST']","['ACLS', 'AMPH', 'CTLT...","['HY', 'MLR', 'OEC']","['ANET', 'ATCO', 'ATKR...",[]
385,2023-03-16,120,"[15, 30, 60, 120]",[],[],[],[],[],"['FTSM', 'SHV']",['SGEN'],[],['MLR'],"['ANET', 'FCN', 'FSLR'...","['ACLS', 'AMPH', 'CTLT...",[],[],[]
386,2023-03-15,120,"[30, 60, 120]",[],[],[],[],[],[],"['FTSM', 'SHV']",[],[],"['CTLT', 'ELF', 'SGEN'...","['ACLS', 'AMPH', 'FCN'...","['ATCO', 'HY', 'LNTH',...","['ATKR', 'BURL', 'FIZZ...",[]


### Get dates in df_close_clean that are missing in df_picks<br>==============================

In [6]:
# earliest date in df_picks
start_date = df_picks.date_end_df_train.min()
# load df with symbols' close
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)
# latest date in df_close_clean
end_date = df_close_clean.index[-1].strftime('%Y-%m-%d')
print(f'start_date, earliest date in df_picks:   {start_date}')
print(f'end_date, latest date in df_close_clean: {end_date}')
print(f'date range to find dates in df_close_clean that are missing in df_picks: {start_date} - {end_date}')

start_date, earliest date in df_picks:   2023-03-15
end_date, latest date in df_close_clean: 2023-12-20
date range to find dates in df_close_clean that are missing in df_picks: 2023-03-15 - 2023-12-20


In [7]:
l_dates_df_picks = df_picks.date_end_df_train.unique().tolist()  # unique dates in df_picks
print(f'l_dates_df_picks, len({len(l_dates_df_picks)}):\n{l_dates_df_picks}')

l_dates_df_picks, len(194):
['2023-12-19', '2023-12-18', '2023-12-15', '2023-12-14', '2023-12-13', '2023-12-12', '2023-12-11', '2023-12-08', '2023-12-07', '2023-12-06', '2023-12-05', '2023-12-04', '2023-12-01', '2023-11-30', '2023-11-29', '2023-11-28', '2023-11-27', '2023-11-24', '2023-11-22', '2023-11-21', '2023-11-20', '2023-11-17', '2023-11-16', '2023-11-15', '2023-11-14', '2023-11-13', '2023-11-10', '2023-11-09', '2023-11-08', '2023-11-07', '2023-11-06', '2023-11-03', '2023-11-02', '2023-11-01', '2023-10-31', '2023-10-30', '2023-10-27', '2023-10-26', '2023-10-25', '2023-10-24', '2023-10-23', '2023-10-20', '2023-10-19', '2023-10-18', '2023-10-17', '2023-10-16', '2023-10-13', '2023-10-12', '2023-10-11', '2023-10-10', '2023-10-09', '2023-10-06', '2023-10-05', '2023-10-04', '2023-10-03', '2023-10-02', '2023-09-29', '2023-09-28', '2023-09-27', '2023-09-26', '2023-09-25', '2023-09-22', '2023-09-21', '2023-09-20', '2023-09-19', '2023-09-18', '2023-09-15', '2023-09-14', '2023-09-13', '2023

In [8]:
# Select rows in df_close_clean between the start_date and end_date in df_picks
mask = (df_close_clean.index >= start_date) & (df_close_clean.index <= end_date)
l_dates_df_close = df_close_clean[mask].index
# list of date index in 'yyyy-mm-dd' format
l_dates_df_close = l_dates_df_close.strftime('%Y-%m-%d')
print(f'l_dates_df_close, (len={len(l_dates_df_close )}):\n{l_dates_df_close }')

l_dates_df_close, (len=195):
Index(['2023-03-15', '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-27', '2023-03-28',
       ...
       '2023-12-07', '2023-12-08', '2023-12-11', '2023-12-12', '2023-12-13', '2023-12-14', '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20'], dtype='object', name='Date', length=195)


In [9]:
dates_missing_in_df_picks  = compare_lists(l_dates_df_close, l_dates_df_picks)
# pickle_dump(dates_missing_in_df_picks, path_data_dump, fp_dates_missing_in_df_picks, verbose=verbose)
print(f'dates_missing_in_df_picks, (len={len(dates_missing_in_df_picks )}):\n{dates_missing_in_df_picks }')

dates_missing_in_df_picks, (len=1):
['2023-12-20']


### ==============================

### Get dates in df_close_clean

In [10]:
# df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)
# Sort the DataFrame by the date index in place
df_close_clean.sort_index(inplace=True)
print(f'df_close_clean:\n{df_close_clean}\n')
df_close_clean_index = df_close_clean.index
print(f'df_close_clean_index:\n{df_close_clean_index}')  


df_close_clean:
                     A         AA        AAL         AAP        AAPL         AB        ABBV        ABR         ABT       ACGL       ACHC       ACIW        ACLS       ACRS        ADBE  ...        XRX         XYL       YELP         YUM       YUMC          YY         ZBH        ZBRA         ZD         ZG       ZION        ZTO         ZTS       ZUMZ        ZWS
Date                                                                                                                                                                                    ...                                                                                                                                                                           
2018-01-05   66.937393  52.944538  51.316174  103.281639   41.370628  15.561795   77.047852   4.778927   53.267544  29.453333  33.110001  22.799999   31.000000  24.629999  185.339996  ...  22.849245   64.364021  43.169998   74.412132  40.453087  111.087791  117.0005

### Given a missing date in df_picks, calculate the number of days_to_drop from df_close_clean s.t. the last date in df_close_clean is the missing date in df_picks

In [11]:
# list to store days_to_drop from df_close_clean
l_days_to_drop = []

# Calculate the number of days to drop from the date index.
for i in range(len(dates_missing_in_df_picks)):
  date = dates_missing_in_df_picks[i]
  last_date_index = df_close_clean_index.get_loc(date)
  # number of days to drop from df such that the last date is a missing date    
  days_to_drop = len(df_close_clean_index) - last_date_index - 1
  l_days_to_drop.append(days_to_drop)  

print(f'dates_missing_in_df_picks, len={len(dates_missing_in_df_picks)}: {dates_missing_in_df_picks}')
print(f'l_days_to_drop, len={len(l_days_to_drop)}: {l_days_to_drop}')

dates_missing_in_df_picks, len=1: ['2023-12-20']
l_days_to_drop, len=1: [0]


### Get picks for the missing dates

In [12]:
df_current = df_close_clean.copy()

# list to store the last df date
l_date_last_index = []

# list to sore date in zipped dates_missing_in_df_picks
l_date_missing_in_df_picks = []

# total number of iteration
i_total = len(l_days_to_drop) * len(l_sorted_days_lookbacks)

for i, values in enumerate(itertools.product(zip(l_days_to_drop, dates_missing_in_df_picks), l_sorted_days_lookbacks)):
    print(f'Start of For_Loop  {i+1} of {i_total} loops{"="*40:>42}')
    drop_last_n_rows = values[0][0]
    date_missing_in_df_picks = values[0][1] 
    days_lookbacks = values[1]
    l_date_missing_in_df_picks.append(date)
 
    print(f'i: {i+1}')    
    print(f'drop_last_n_rows: {drop_last_n_rows}') 
    print(f'date_missing_in_df_picks: {date_missing_in_df_picks}')
    print(f'days_lookbacks: {days_lookbacks}\n')
    print(f'{i+1} of {len(l_sorted_days_lookbacks)} days_lookbacks: {days_lookbacks} in l_sorted_days_lookbacks: {l_sorted_days_lookbacks}')    

    # drops df rows by drop_last_n_rows, limits df length to max_days_lookbacks 
    # e.g. days_lookbacks: [15, 30, 60, 120] => max_days_lookbacks: 120
    max_days_lookbacks = max(days_lookbacks)
    print(f"max_days_lookbacks: {max_days_lookbacks}\n")
    slice_start = -(max_days_lookbacks + drop_last_n_rows)
    slice_end = -drop_last_n_rows
    if drop_last_n_rows == 0:  # return df with all rows
        df = df_current[slice_start:].copy()
    else:  # return df with dropped drop_last_n_rows rows
        df = df_current[slice_start:slice_end].copy()
    print(f"dropped last {drop_last_n_rows} row(s) from df since drop_last_n_rows = {drop_last_n_rows}")
    print(f"df.head(1):\n{df.head(1)}\n")
    print(f"df.tail(1):\n{df.tail(1)}\n")

    date_last_index = df.index[-1].strftime('%Y-%m-%d')
    print(f'date_last_index: {date_last_index}')

    # Error check
    if date_missing_in_df_picks == date_last_index:
        print(f"Passed Error Check: date_missing_in_df_picks {date_missing_in_df_picks} == date_last_index {date_last_index}")  
    else:
        print("ERROR: date_missing_in_df_picks != date_last_index")
        print(f'date_missing_in_df_picks: {date_missing_in_df_picks}')
        print(f'date_last_index: {date_last_index}')
        sys.exit(1)  

    l_date_last_index.append(date_last_index)
    len_df = len(df)
    len_slice = slice_end - slice_start
    # print(f"len(df): {len(df)}\n")
    print(f"len_df: {len_df}, len_slice: {len_slice}\n")    

    # Since df rows has been sliced to max_days_lookbacks,
    #   n_samples > 1, will always return more copies of the same slice. 
    # Returns a list of random tuples of start_train, end_train, end_eval,
    # where iloc[start_train:end_train] is used for training,
    # and iloc[end_train:end_eval] is used for evaluation.  The length of the
    # list is equal to n_samples.
    max_lookback_slices = random_slices(
        len_df = len_df,
        n_samples=n_samples,
        days_lookback=max(days_lookbacks),
        days_eval=days_eval,
        verbose=False,
    )    

    # Create sets of sub-slices from max_slices and days_lookbacks. A slice is
    # a tuple of iloc values for start_train:end_train=start_eval:end_eval.
    # Given 2 max_slices of [(104, 224, 234), (626, 746, 756)], it returns 2 sets
    # [[(194, 224, 234), (164, 224, 234), (104, 224, 234)],
    # [(716, 746, 756), (686, 746, 756), (626, 746, 756)]]. End_train is constant
    # for each set. End_train - start_train is the value of the maximum slice.     
    sets_lookback_slices = lookback_slices(
        max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False
    )

    if verbose:
        print(f"number of random samples of max_lookback_slices taken is n_samples = {n_samples}")
        print(f"max_lookback_slices: {max_lookback_slices}\n")
        print(f"days_lookbacks: {days_lookbacks}")
        print(f"sets_lookback_slices, e.g. (start_train:end_train:end_eval): {sets_lookback_slices}\n")

        print(f"number of sets in sets_lookback_slices is equal to n_samples = {n_samples}")
        print(
            f'number of tuples in each "set of lookback slices" is equal to len(days_lookbacks): {len(days_lookbacks)}'
        )

    # If given:
    #  performance metric: r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
    #  l_sorted_days_lookbacks: [[30, 60, 120], [15, 30, 60, 120]]
    #   => days_lookbacks: [30, 60, 120]
    #    => sets_lookback_slices: [[(90, 120, 120), (60, 120, 120), (0, 120, 120)]]
    # Then, grp_top_set_syms_n_freq is a list of lists of the top n_top_syms of the
    # best performing symbols and their number of occurrence for sets_lookback_slices.
    # The list of lists corresponds to days_lookbacks in l_sorted_days_lookbacks.  
    #  e.g. grp_top_set_syms_n_freq:
    #   [[('GPS', 8), ('SHV', 8), ('FTSM', 7), ('GBTC', 7), ('BTC-USD', 6), ('CBOE', 6), ('ANF', 5), ('NRG', 5), ('WING', 5), ('DELL', 4), ('EDU', 4), ('HIBB', 4), ('LRN', 4), ('ALL', 3), ('CAH', 3), ('CMG', 3), ('GDDY', 3), ('HRB', 3), ('MDLZ', 3), ('PGR', 3)]]
    # grp_top_set_syms is grp_top_set_syms_n_freq with number of occurrence dropped
    #   e.g. [['GPS', 'SHV', 'FTSM', 'GBTC', 'BTC-USD', 'CBOE', 'ANF', 'NRG', 'WING', 'DELL']]
    # date_end_df_train, e.g. 2023-11-22
    (
        grp_top_set_syms_n_freq,
        grp_top_set_syms,
        date_end_df_train,
    ) = best_perf_syms_sets_lookback_slices(
        df_close=df,
        sets_lookback_slices=sets_lookback_slices,
        n_top_syms=20,
        syms_start=0,
        syms_end=10,
        verbose=verbose
        )

    print(f'\nOutput from function best_perf_syms_sets_lookback_slices')
    print(f'{"`"*60}')
    print(f'sets_lookback_slices: {sets_lookback_slices}\n')
    print(f'grp_top_set_syms_n_freq:\n{grp_top_set_syms_n_freq}\n')
    print(f'grp_top_set_syms:\n{grp_top_set_syms}\n')
    print(f'date_end_df_train:\n{date_end_df_train}')    
    print(f'{"`"*60}\n\n')

    for j, top_set_syms_n_freq in enumerate(grp_top_set_syms_n_freq):
        # If given top_set_syms_n_freq:
        #  [('GPS', 10), ('SHV', 9), ('FTSM', 7), ('GBTC', 7), ('WING', 7), ('CBOE', 6),
        #  ('ANF', 5), ('BTC-USD', 5), ('NRG', 5), ('TSEM', 5), ('BURL', 4), ('CRSP', 4),
        #  ('EDU', 4), ('GDDY', 4), ('LRN', 4), ('NFLX', 4), ('PI', 4), ('WRB', 4),
        #  ('AXGN', 3), ('CAH', 3)]
        # Then, l_sym_freq_cnt, where symbol frequency count is from 15, 14, ..., 2:
        #  [[], [], [], [], [], ['GPS'], ['SHV'], [], ['FTSM', 'GBTC', 'WING'], ['CBOE'],
        #  ['ANF', 'BTC-USD', 'NRG', 'TSEM'],
        #  ['BURL', 'CRSP', 'EDU', 'GDDY', 'LRN', 'NFLX', 'PI', 'WRB'],
        #  ['AXGN', 'CAH'], []]
        l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
        print(f'{j}, grp_top_set_syms_n_freq:\n{grp_top_set_syms_n_freq}')
        print(f'{j}, top_set_syms_n_freq:\n{top_set_syms_n_freq}')
        print(f'{j}, l_sym_freq_cnt:\n{l_sym_freq_cnt}\n')                
        
        if verbose:
            print(f"set_lookback_slices: {sets_lookback_slices[j]}")
            print(f"max_lookback_slices: {max_lookback_slices}\n")
            print(f"data below will be added to {fp_df_picks}")
            print(f"date_end_df_train:   {date_end_df_train}")
            print(f"max_days_lookbacks:  {max_days_lookbacks}")
            print(f"days_lookbacks:      {days_lookbacks}")
            print(f"sym_freq_15:         {l_sym_freq_cnt[0]}")
            print(f"sym_freq_14:         {l_sym_freq_cnt[1]}")
            print(f"sym_freq_13:         {l_sym_freq_cnt[2]}")
            print(f"sym_freq_12:         {l_sym_freq_cnt[3]}")
            print(f"sym_freq_11:         {l_sym_freq_cnt[4]}")
            print(f"sym_freq_10:         {l_sym_freq_cnt[5]}")
            print(f"sym_freq_9:          {l_sym_freq_cnt[6]}")
            print(f"sym_freq_8:          {l_sym_freq_cnt[7]}")
            print(f"sym_freq_7:          {l_sym_freq_cnt[8]}")
            print(f"sym_freq_6:          {l_sym_freq_cnt[9]}")
            print(f"sym_freq_5:          {l_sym_freq_cnt[10]}")
            print(f"sym_freq_4:          {l_sym_freq_cnt[11]}")
            print(f"sym_freq_3:          {l_sym_freq_cnt[12]}")
            print(f"sym_freq_2:          {l_sym_freq_cnt[13]}\n")

    if store_results:
        row_picks0 = [date_end_df_train, max_days_lookbacks, str(days_lookbacks)]
        row_picks1 = [
            str(l_sym_freq_cnt[0]),
            str(l_sym_freq_cnt[1]),
            str(l_sym_freq_cnt[2]),
            str(l_sym_freq_cnt[3]),
        ]
        row_picks2 = [
            str(l_sym_freq_cnt[4]),
            str(l_sym_freq_cnt[5]),
            str(l_sym_freq_cnt[6]),
            str(l_sym_freq_cnt[7]),
        ]
        row_picks3 = [
            str(l_sym_freq_cnt[8]),
            str(l_sym_freq_cnt[9]),
            str(l_sym_freq_cnt[10]),
            str(l_sym_freq_cnt[11]),
        ]
        row_picks4 = [str(l_sym_freq_cnt[12]), str(l_sym_freq_cnt[13])]
        row_picks_total = row_picks0 + row_picks1 + row_picks2 + row_picks3 + row_picks4
        print(f"row_picks_total: {row_picks_total}")

        df_picks.loc[len(df_picks)] = row_picks_total
        pickle_dump(df_picks, path_data_dump, fp_df_picks)
        print(f"appended row_picks_total to df_picks:\n{row_picks_total}\n")

    print(f'End of For_Loop  {i+1} of {i_total} loops{"="*40:>44}\n\n')    

Start of For_Loop  1 of 2 loops  ========================================
i: 1
drop_last_n_rows: 0
date_missing_in_df_picks: 2023-12-20
days_lookbacks: [30, 60, 120]

1 of 2 days_lookbacks: [30, 60, 120] in l_sorted_days_lookbacks: [[30, 60, 120], [15, 30, 60, 120]]
max_days_lookbacks: 120

dropped last 0 row(s) from df since drop_last_n_rows = 0
df.head(1):
                    A        AA   AAL        AAP        AAPL         AB        ABBV        ABR         ABT       ACGL       ACHC       ACIW        ACLS   ACRS        ADBE  ...        XRX         XYL       YELP         YUM       YUMC         YY         ZBH        ZBRA         ZD         ZG       ZION    ZTO         ZTS   ZUMZ        ZWS
Date                                                                                                                                                                        ...                                                                                                                              